# Spatial Selection

In [3]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.append("..")   

# Importing fields and counties
counties = gpd.read_file("/workspace/workflow/counties.geojson")
counties = counties[['name','geometry']]
counties.columns=['countyname','geometry']

fields=gpd.read_file("/workspace/workflow/_2GridSampling/field_final_sample.geojson")
fields=fields[['id_cell','id_within_cell','geometry']]

# Mapping Region
region3_map = {
    # NW
    "Jasper County": "NC", "Lake County": "NC", "Laporte County": "NC", "Newton County": "NC", "Porter County": "NC", "Pulaski County": "NC", "Starke County": "NC", "White County": "NC",
    # NC
    "Kosciusko County":"NC","Wabash County":"NC","St Joseph County":"NC","Cass County": "NC", "Fulton County": "NC", "Howard County": "C", "Miami County": "NC", "Tippecanoe County": "NC", "Tipton County": "C", "Carroll County": "NC", "Clinton County": "C","Marshall County":"NC","Elkhart County":"NC",
    # NE
    "Adams County": "NE", "Allen County": "NE", "Dekalb County": "NE", "Huntington County": "NE", "Lagrange County": "NE", "Noble County": "NE", "Steuben County": "NE", "Wells County": "NE", "Whitley County": "NE",
    # WC
    "Benton County": "NC", "Fountain County": "NC", "Montgomery County": "NC", "Parke County": "NC", "Putnam County": "NC", "Vermillion County": "NC", "Warren County": "NC","Vigo County": "NC","Clay County": "NC",
    # C
    "Boone County": "C", "Hamilton County": "C", "Hancock County": "C", "Hendricks County": "C", "Johnson County": "C", "Madison County": "C", "Marion County": "C", "Morgan County": "C", "Shelby County": "C",
    # EC
    "Blackford County": "NE","Union County": "NE","Fayette County": "NE", "Delaware County": "NE", "Grant County": "C", "Henry County": "NE", "Jay County": "NE", "Randolph County":"NE","Rush County":"C","Wayne County":"NE",
    
    "Daviess County": "NC", "Sullivan County": "NC","Gibson County": "NC", "Knox County": "NC", "Perry County": "NC", "Pike County": "NC", "Posey County": "NC", "Spencer County": "NC", "Vanderburgh County": "NC", "Warrick County": "NC",
    # SC
    "Brown County": "NC", "Crawford County": "NC", "Dubois County": "NC", "Greene County": "NC", "Lawrence County": "NC", "Martin County": "NC", "Monroe County": "NC", "Orange County": "NC", "Owen County": "NC", "Washington County": "NC",
    # SE
    "Bartholomew County": "C", "Clark County": "NC", "Decatur County": "C", "Dearborn County": "NC", "Floyd County": "NC", "Franklin County": "NC", "Harrison County": "NC", "Jackson County": "NC", "Jefferson County": "NC", "Jennings County": "NC", "Ohio County": "NC", "Ripley County": "NC", "Scott County": "NC", "Switzerland County": "NC"
}

# region9_map = {
#     # NW
#     "Jasper County": "NW", "Lake County": "NW", "Laporte County": "NW", "Newton County": "NW", "Porter County": "NW", "Pulaski County": "NW", "Starke County": "NW", "White County": "NW",
#     # NC
#     "Kosciusko County":"NC","Wabash County":"NC","St Joseph County":"NC","Cass County": "NC", "Fulton County": "NC", "Howard County": "C", "Miami County": "NC", "Tippecanoe County": "WC", "Tipton County": "C", "Carroll County": "NC", "Clinton County": "C","Marshall County":"NC","Elkhart County":"NC",
#     # NE
#     "Adams County": "NE", "Allen County": "NE", "Dekalb County": "NE", "Huntington County": "NE", "Lagrange County": "NE", "Noble County": "NE", "Steuben County": "NE", "Wells County": "NE", "Whitley County": "NE",
#     # WC
#     "Benton County": "NW", "Fountain County": "WC", "Montgomery County": "WC", "Parke County": "WC", "Putnam County": "WC", "Vermillion County": "WC", "Warren County": "WC","Vigo County": "WC","Clay County": "WC",
#     # C
#     "Boone County": "C", "Hamilton County": "C", "Hancock County": "C", "Hendricks County": "C", "Johnson County": "C", "Madison County": "C", "Marion County": "C", "Morgan County": "C", "Shelby County": "C",
#     # EC
#     "Blackford County": "EC","Union County": "EC","Fayette County": "EC", "Delaware County": "EC", "Grant County": "C", "Henry County": "EC", "Jay County": "EC", "Randolph County": "EC", "Rush County": "C", "Wayne County": "EC",
#     # SW
#     "Daviess County": "SW", "Sullivan County": "SW","Gibson County": "SW", "Knox County": "SW", "Perry County": "SC", "Pike County": "SW", "Posey County": "SW", "Spencer County": "SW", "Vanderburgh County": "SW", "Warrick County": "SW",
#     # SC
#     "Brown County": "SC", "Crawford County": "SC", "Dubois County": "SW", "Greene County": "SW", "Lawrence County": "SC", "Martin County": "SW", "Monroe County": "SC", "Orange County": "SC", "Owen County": "WC", "Washington County": "SC",
#     # SE
#     "Bartholomew County": "C", "Clark County": "SE", "Decatur County": "C", "Dearborn County": "SE", "Floyd County": "SC", "Franklin County": "SE", "Harrison County": "SC", "Jackson County": "SC", "Jefferson County": "SE", "Jennings County": "SE", "Ohio County": "SE", "Ripley County": "SE", "Scott County": "SE", "Switzerland County": "SE"
# }

counties["region"] = counties["countyname"].map(region3_map).fillna("Other/SandNI")
counties=counties[["region","countyname","geometry"]]

fields = fields.to_crs(counties.crs)

region_fields = gpd.sjoin(fields, counties, how='inner', predicate='intersects')

region_fields_sample = (
    region_fields.groupby('region', group_keys=False) 
       .apply(lambda x: x.sample(n=10, random_state=42)) 
)
region_fields_sample.reset_index(drop=True, inplace=True)

region_fields_sample.to_file("/workspace/workflow/_3AgroDataExtraction/calibration_sample.geojson", driver="GeoJSON")


/tmp/ipykernel_16655/2702864584.py:67: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=10, random_state=42))


# Features Extraction

In [4]:
!python -m _3AgroDataExtraction

1980-01-01T00:00:00
2024-12-31T00:00:00
Transforming shp to WGS 84
Note: Kf and Kw have numeric values at the chorizon level,
they are really categorical data and only available to the minmax aggegation method
Note: Kf and Kw have numeric values at the chorizon level,
they are really categorical data and only available to the minmax aggegation method
Note: Kf and Kw have numeric values at the chorizon level,
they are really categorical data and only available to the minmax aggegation method
Note: Kf and Kw have numeric values at the chorizon level,
they are really categorical data and only available to the minmax aggegation method
Note: Kf and Kw have numeric values at the chorizon level,
they are really categorical data and only available to the minmax aggegation method
Note: Kf and Kw have numeric values at the chorizon level,
they are really categorical data and only available to the minmax aggegation method
Note: Kf and Kw have numeric values at the chorizon level,
they are really 

In [5]:
!python -m _4preproccesing

2021-01-01T00:00:00
2024-12-31T00:00:00
2021-01-01T00:00:00
2024-12-31T00:00:00
2021-01-01T00:00:00
2024-12-31T00:00:00
2021-01-01T00:00:00
2024-12-31T00:00:00
2021-01-01T00:00:00
2024-12-31T00:00:00
2021-01-01T00:00:00
2024-12-31T00:00:00
2021-01-01T00:00:00
2024-12-31T00:00:00
2021-01-01T00:00:00
2024-12-31T00:00:00
2021-01-01T00:00:00
2024-12-31T00:00:00
2021-01-01T00:00:00
2024-12-31T00:00:00
2021-01-01T00:00:00
2024-12-31T00:00:00
2021-01-01T00:00:00
2024-12-31T00:00:00
2021-01-01T00:00:00
2024-12-31T00:00:00
2021-01-01T00:00:00
2024-12-31T00:00:00
2021-01-01T00:00:00
2024-12-31T00:00:00
2021-01-01T00:00:00
2024-12-31T00:00:00
2021-01-01T00:00:00
2024-12-31T00:00:00
2021-01-01T00:00:00
2024-12-31T00:00:00
2021-01-01T00:00:00
2024-12-31T00:00:00
2021-01-01T00:00:00
2024-12-31T00:00:00
2021-01-01T00:00:00
2024-12-31T00:00:00
2021-01-01T00:00:00
2024-12-31T00:00:00
2021-01-01T00:00:00
2024-12-31T00:00:00
2021-01-01T00:00:00
2024-12-31T00:00:00
2021-01-01T00:00:00
2024-12-31T00:00:00


In [6]:
!python -m _5RunSimulations

2021-01-01T00:00:00
2023-12-31T00:00:00
Simulation successful!
Simulation successful!
Simulation successful!
2021-01-01T00:00:00
2023-12-31T00:00:00
Simulation successful!
Simulation successful!
Simulation successful!
2021-01-01T00:00:00
2023-12-31T00:00:00
Simulation successful!
Simulation successful!
Simulation successful!
2021-01-01T00:00:00
2023-12-31T00:00:00
Simulation successful!
Simulation successful!
Simulation successful!
2021-01-01T00:00:00
2023-12-31T00:00:00
Simulation successful!
Simulation successful!
Simulation successful!
2021-01-01T00:00:00
2023-12-31T00:00:00
Simulation successful!
Simulation successful!
Simulation successful!
2021-01-01T00:00:00
2023-12-31T00:00:00
Simulation successful!
Simulation successful!
Simulation successful!
2021-01-01T00:00:00
2023-12-31T00:00:00
Simulation successful!
Simulation successful!
Simulation successful!
2021-01-01T00:00:00
2023-12-31T00:00:00
Simulation successful!
Simulation successful!
Simulation successful!
2021-01-01T00:00:00